In [5]:
pip install import-ipynb

  Using cached import_ipynb-0.1.4-py3-none-any.whl (4.1 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import music21 as m21
import json
import tensorflow as tf
import keras
import numpy as np
# from main import LENGTH, DICT_PATH
import import_ipynb
# from ipynb.fs.full.Preprocessing import training_sequence, LENGTH
from Preprocessing import training_sequence, LENGTH, DICT_PATH

importing Jupyter notebook from Preprocessing.ipynb


In [11]:
class MelodyGenerator:

    # "misc/Network_model.h5"
    def __init__(self, model_path="NetworkModel.h5"):
    # def __init__(self, model_path="misc/Network_model.h5"):

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(DICT_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * LENGTH


    def generateMelody(self, seed, num_steps, max_sequence_length, temperature):

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # (1, max_sequence_length, num of symbols in the vocabulary)
            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self._sample_with_temperature(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def _sample_with_temperature(self, probabilites, temperature):

        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        # print(probabilites.flatten())

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        # index = np.random.choice(choices, p=probabilites)
        index = np.random.choice(choices, p=probabilites.flatten())

        return index
    
    def saveMelody(self,melody, step_duration=0.25, format="midi", file_name="melody_1_something.mid"): # "melody_1.mid"
        # step_duration is the amount of duration in quarter note length that we have at each step in out time series representation.

        # create a stream
        stream = m21.stream.Stream()

        # Pasring through all the symbols in melody and creating note/rest objects

        start_symbol = None
        counter = 1
    
        for i, symbol in enumerate(melody):

            if symbol != "_" or i + 1 == len(melody):
            # Make sure that the first event is not being dealt
                if start_symbol is not None:

                    quarter_length_duration = step_duration * counter

                    if start_symbol == "r":
                        event = m21.note.Rest(quarterLength=quarter_length_duration)
                    else:
                        event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)
                
                    stream.append(event)
                    # rest counter
                    counter = 1

                start_symbol = symbol

            else:
                counter += 1
                
    
        stream.write(format, file_name)

In [12]:
seed = "64 _ 69 _ _ _ 71 _ 72 _ _ 71 "
g = MelodyGenerator()
melody = g.generateMelody(seed, 500, LENGTH, 0.8)

print(melody)
g.saveMelody(melody)

2023-03-20 08:46:12.139620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-20 08:46:12.178563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 13ms/step
['64', '_', '69', '_', '_', '_', '71', '_', '72', '_', '_', '71', '51']


In [ ]:
seed = "64 _ 69 _ _ _ 71 _ 72 _ _ 71 "
g = MelodyGenerator()
melody = g.generateMelody(seed, 500, LENGTH, 0.8)

print(melody)
g.saveMelody(melody)

1/1 [==============================] - 0s 17ms/step


2023-03-04 10:04:25.903454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-04 10:04:25.952157: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 17ms/step
['64', '_', '69', '_', '_', '_', '71', '_', '72', '_', '_', '71', '51', '57', '51']


In [10]:
seed = "64 _ 69 _ _ _ 71 _ 72 _ _ 71 "
g = MelodyGenerator()
melody = g.generateMelody(seed, 500, LENGTH, 0.8)

print(melody)
g.saveMelody(melody)

1/1 [==============================] - 1s 551ms/step


2023-03-20 08:41:00.094766: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-20 08:41:00.185842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-20 08:41:00.197228: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-20 08:41:00.241872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 25ms/step


IndexError: list index out of range

In [ ]:
seed = "64 _ 69 _ _ _ 71 _ 72 _ _ 71 "
g = MelodyGenerator()
melody = g.generateMelody(seed, 500, LENGTH, 0.8)

print(melody)
g.saveMelody(melody)

1/1 [==============================] - 0s 276ms/step


2023-03-03 09:53:53.390563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:53:53.438374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


None
1/1 [==============================] - 0s 267ms/step


2023-03-03 09:53:53.692855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:53:53.741336: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


None
1/1 [==============================] - 1s 520ms/step


...

2023-03-03 09:53:59.157881: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:53:59.205052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


None
1/1 [==============================] - 0s 265ms/step


2023-03-03 09:53:59.452902: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:53:59.499966: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


None
1/1 [==============================] - 0s 266ms/step
None
['64', '_', '69', '_', '_', '_', '71', '_', '72', '_', '_', '71', '51', '57', '59', '78', '55', '51', '59', '59', '55', '76', '55', '78', '59', '57', '51', '62', '59', '57', '57', '78']


2023-03-03 09:53:59.750833: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-03 09:53:59.798380: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
